In [9]:
from pynq.overlays.base import BaseOverlay 
import threading
import time
import asyncio 
import random
import sys
from datetime import datetime

In [10]:
base = BaseOverlay("base.bit") 
btns = base.btns_gpio

In [11]:
%%microblaze base.PMODA 
#include "gpio.h"
#include "pyprintf.h"
//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}
//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

In [12]:
class led:
    def __init__(self):
        write_gpio(3, 0)
        write_gpio(2, 0)
        write_gpio(1, 0)
        
    def off(self):
        write_gpio(2, 0)
        
    def on(self):
        write_gpio(2, 1)
        
        
class phil:
    def __init__(self,led,i,forkLeft,forkRight):
        self.num_of_forks = 0
        self.led = led
        self.index = i
        self.forkLeft = forkLeft
        self.forkRight = forkRight
        print(f"Seating Phil{self.index}")
        self.nap_time = random.randint(3, 10)
        self.eat_time = self.nap_time * random.randint(2, 5)

    def starving(self):
        print(f"Phil{self.index} is starving")
        self.led.off()
        #Starving, they are waiting for the forks
        #Should they be trying to claim a fork such that num_of_fork ==1?
    
    def eating(self): #quicker blinking than napping
        print(f"Phil{self.index} is eating")
        for i in range(self.eat_time):
            self.led.on()
            time.sleep(0.1)
            self.led.off()
            time.sleep(0.1)
        print(f"Phil{self.index} finished eating")
        #Eating they have both the forks
        #So num_of_forks == 2
    
    def napping(self):
        print(f"Phil{self.index} is napping")
        for i in range(self.nap_time):    
            self.led.on()
            time.sleep(0.4)
            self.led.off()
            time.sleep(0.4)
        print(f"Phil{self.index} finished napping")
        #Napping, they are not actively looking for forks
        #So num_of_forks should be == 0 to be efficient with forks      


In [13]:
feeding = True
      
#what is async python def
def get_btns(): 
    global feeding
    time.sleep(0.01)
    while feeding:
        if btns.read() == 1: 
            feeding = False
            print("BTN pressed")
            sys.exit()
    
    
def settable():
    print("Setting Table")
    
    p0 = base.leds[0]
    p1 = base.leds[1]
    p2 = base.leds[2]
    p3 = base.leds[3]
    p4 = led()
    print("Philosphers here")
    
    mutex = threading.Lock()
            
    
    f0 = threading.Semaphore()
    f1 = threading.Semaphore()
    f2 = threading.Semaphore()
    f3 = threading.Semaphore()
    f4 = threading.Semaphore()
    print("Forks procured")
    
    l_forks = [f0, f1, f2, f3, f4]
    l_phil = [phil(p0,0,f0,f1),phil(p1,1,f1,f2),phil(p2,2,f2,f3),phil(p3,3,f3,f4),phil(p4,4,f4,f0)]
    
    
    threads = []
    
    
    for i in range(5):
        t = threading.Thread(target=feedphils, args=(l_phil[i],mutex,))
        #t = threading.Thread(target=pri, args=(1,))
        threads.append(t)
    
    t = threading.Thread(target=get_btns, args=())
    threads.append(t)
    
    print("Starting Threads")
    for t in threads:
        t.start()
        
    #what is the point of this
    for t in threads:
        t.join()
    
#Why cant a thread be async? removed async to have the function be called by the thread
def feedphils(phil,mutex):
    # I think I am assuming that napping is longer than eating
    global feeding

    while feeding:
        #inital conditions. All Starving and want the forks. how to distribute
        # Condition where there are 2 or more forks, Always ensure give two forks
        # If one fork left and we gave out forks in pairs. 
        # One phil will have 1 fork and the other two will have none and will be next up 
        # After the inital conditions are sorted out then it is the steady state one.
        lookingForFork = False
        
        #lookingForFork = mutex.acquire(True)
        #if lookingForFork:
        #    print(f"Phil{phil.index} has the mutex")
           
        haveLeftFork = phil.forkLeft.acquire(False)
        haveRightFork = phil.forkRight.acquire(False)
        
        if haveLeftFork:
            print(f"Phil{phil.index} has left fork")
        if haveRightFork: 
            print(f"Phil{phil.index} has right fork")
        
        #if haveLeftFork and haveRightFork and lookingForFork:
        if haveLeftFork and haveRightFork:
            
            phil.eating()
            
            phil.forkLeft.release()
            print(f"Phil{phil.index} has released left fork after eating")
            
            phil.forkRight.release()
            print(f"Phil{phil.index} has released right fork after eating")
            
            if lookingForFork:
                try:
                    mutex.release()
                    print(f"Phil{phil.index} is releasing mutex after forks")
                except:
                    print(f"Phil{phil.index} error. haveLeftFork: {haveLeftFork} haveRightFork: {haveRightFork} lookingForFork:{lookingForFork}")
                
            phil.napping()
            phil.starving()
        else:
            if haveLeftFork:
                phil.forkLeft.release()
                print(f"Phil{phil.index} has released left fork")
            if haveRightFork:
                phil.forkRight.release()
                print(f"Phil{phil.index} has released right fork")
                

        if lookingForFork: 
            print(f"Phil{phil.index} is releasing mutex")
            mutex.release()
        time.sleep(0.1)

In [14]:
print("Start")
#loop = asyncio.new_event_loop() #what is this code doing
#loop.create_task(settable())  
#loop.run_forever()
#loop.close()
feeding = True
settable()
print("Done")


Start
Setting Table
Philosphers here
Forks procured
Seating Phil0
Seating Phil1
Seating Phil2
Seating Phil3
Seating Phil4
Starting Threads
Phil0 has left forkPhil1 has right fork
Phil1 has released right fork

Phil0 has right fork
Phil0 is eating
Phil2 has left forkPhil3 has right fork
Phil2 has right fork
Phil2 is eating

Phil3 has released right fork
Phil4 has left fork
Phil4 has released left fork
Phil3 has right fork
Phil3 has released right fork
Phil4 has left fork
Phil4 has released left fork
Phil3 has right fork
Phil3 has released right fork
Phil4 has left fork
Phil4 has released left fork
Phil3 has right fork
Phil3 has released right fork
Phil4 has left fork
Phil4 has released left fork
Phil3 has right fork
Phil3 has released right forkPhil4 has left fork

Phil4 has released left fork
Phil3 has right fork
Phil3 has released right fork
Phil4 has left fork
Phil4 has released left fork
Phil3 has right fork
Phil3 has released right fork
Phil4 has left fork
Phil4 has released left f

TypeError: __init__() missing 3 required positional arguments: 'i', 'forkLeft', and 'forkRight'

In [21]:
p.napping()
